# RCS CICAG workshop &mdash; KLIFS: making kinase structures work

## Aim of this notebook

[KLIFS](https://klifs.net/) is a database for kinase-ligand interaction fingerprints and structures. In this notebook, we will use the programmatic access to this database ([KLIFS OpenAPI](https://klifs.net/swagger_v2/)) and the [OpenCADD-KLIFS](https://github.com/volkamerlab/opencadd) package to interact with its rich content. 

We will assess the similarity between a set of kinases based on interaction fingerprints (KLIFS IFP) and subpocket-based structural fingerprints (KiSSim fingerprint) - a short demo of [TeachOpenCADD's kinase similarity edition](https://projects.volkamerlab.org/teachopencadd/talktorials.html#kinase-similarity) which was set up in collaboration with Talia B. Kimber.

## References

The notebook is a mix of the following [TeachOpenCADD](https://academic.oup.com/nar/advance-article/doi/10.1093/nar/gkac267/6582172) notebooks:
- [T012 · Data acquisition from KLIFS](https://projects.volkamerlab.org/teachopencadd/talktorials/T012_query_klifs.html)
- [T025 · Kinase similarity: Kinase pocket (KiSSim fingerprint)](https://projects.volkamerlab.org/teachopencadd/talktorials/T025_kinase_similarity_kissim.html)
- [T026 · Kinase similarity: Interaction fingerprints](https://projects.volkamerlab.org/teachopencadd/talktorials/T026_kinase_similarity_ifp.html)
- [T028 · Kinase similarity: Compare different perspectives](https://projects.volkamerlab.org/teachopencadd/talktorials/T028_kinase_similarity_compare_perspectives.html)

We are using the following open-source resources:
- KLIFS database &mdash; a structural kinase database: [Website](https://klifs.net) and [paper](https://doi.org/10.1093/nar/gkaa895)
- OpenCADD-KLIFS &mdash; a Python module to fetch KLIFS data: [Code](github.com/volkamerlab/opencadd) and [paper](https://joss.theoj.org/papers/10.21105/joss.03951) and [documentation](https://opencadd.readthedocs.io/)
- KiSSim &mdash; a KLIFS-based kinase structural similarity fingerprint: [Code](github.com/volkamerlab/kissim) and [paper](https://pubs.acs.org/doi/abs/10.1021/acs.jcim.2c00050) and [documentation](https://kissim.readthedocs.io/)

## Installation (Google Colab)

In [1]:
# If the notebook is run on Google Colab
# install condacolab and kissim
try:
    import google.colab
    !pip install condacolab
    import condacolab
    condacolab.install()
    !mamba install -yq kissim
except ModuleNotFoundError:
    pass

## Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Define kinase set

In [3]:
kinase_names = ["EGFR", "ErbB2", "SLK", "p38a", "p110a"]

## Generate a KLIFS Python client (KLIFS OpenAPI)

- The KLIFS database offers standardized URL schemes (__REST API__) to programmatically access resources that live on the KLIFS server.
- KLIFS defines how these URL have to look like in its REST API scheme ([__OpenAPI specification__](https://swagger.io/docs/specification/about/), formerly __Swagger specification__). 
  - Take a look at how such a document looks like in case of KLIFS (it's a json file): https://klifs.net/swagger/swagger.json 
  - You can also explore the definitions via an interactive user interface (Swagger UI): http://klifs.net/swagger/
- Libraries like `bravado` can be used to dynamically generate a Python client based on OpenAPI definitions. 

In [4]:
from bravado.client import SwaggerClient

In [5]:
KLIFS_API_DEFINITIONS = "https://klifs.net/swagger/swagger.json"
KLIFS_CLIENT = SwaggerClient.from_url(
    KLIFS_API_DEFINITIONS, config={"validate_responses": False}
)

In [ ]:
KLIFS_CLIENT.Information.get_kinase_ID(
    kinase_name="EGFR", species="Human"
).response().result

In [ ]:
KLIFS_CLIENT.Structures.get_structures_pdb_list(
    pdb_codes=["3w32", "3poz"]
).response().result

## Set up a remote KLIFS session with OpenCADD-KLIFS

- `opencadd` is a Python library for structural cheminformatics.
- The submodule `opencadd.databases.klifs` (OpenCADD-KLIFS) offers a standardized API to work with KLIFS data locally (KLIFS download) or remotely (KLIFS OpenAPI).
- Most query results are returned in the form of standardized `pandas` DataFrames for quick and easy data manipulation.

![OpenCADD-KLIFS](https://raw.githubusercontent.com/volkamerlab/opencadd/master/paper/opencadd_klifs_toc.png)

In [ ]:
from opencadd.databases.klifs import setup_remote

session = setup_remote()
pd.set_option("display.max_columns", None)

## Get kinase KLIFS IDs from kinase names

In [ ]:
kinases = session.kinases.by_kinase_name(
    kinase_names=kinase_names, species="Human"
)
kinases

In [ ]:
kinase_klifs_ids = kinases["kinase.klifs_id"].to_list()
print("Kinase KLIFS IDs:", *kinase_klifs_ids)

## Define structure set

Fetch and filter structures that represent our kinase set.

### Fetch structures for kinase set

In [ ]:
print("Kinase names:", *kinase_names)

In [ ]:
structures_df = session.structures.by_kinase_name(kinase_names=kinase_names)
structures_df = structures_df.drop("interaction.fingerprint", axis=1)
print(f"Number of structures: {len(structures_df)}")
print("Kinases:", *structures_df["kinase.klifs_name"].unique())

Let’s have a look at what is stored in the structures’ DataFrame:

In [ ]:
structures_df.head()

### Filter structures

We filter the structures by different criteria:

- Species: human
- Conformation: DFG-in (the active kinase conformation)
- Resolution: $\le 3$ Angström
- Quality score*: $\ge 6$
- Ligand-bound (ligand KLIFS ID cannot be $0$)

\* The KLIFS quality score takes into account the quality of the alignment, as well as the number of missing residues and atoms. A higher score indicates a better structure quality.

In [ ]:
structures_df = structures_df[
    (structures_df["species.klifs"] == "Human")
    & (structures_df["structure.dfg"] == "in")
    & (structures_df["structure.resolution"] <= 3)
    & (structures_df["structure.qualityscore"] >= 6)
    & (structures_df["ligand.klifs_id"] != 0)
]
print(f"Number of structures: {len(structures_df)}")
print("Kinases:", *structures_df["kinase.klifs_name"].unique())

In [ ]:
structures_df.groupby("kinase.klifs_name").size().sort_values(ascending=False)

Usually, we would use all structures for our kinase assessment; to save some computing time in this demo, we will reduce the number of structures per kinase.

In [ ]:
n_structures_per_kinase = 3

In [ ]:
# Sort structures by kinase and quality
structures_df = structures_df.sort_values(
    by=["kinase.klifs_name", "structure.resolution", "structure.qualityscore"],
    ascending=[True, True, False],
)
# Reduce number of structures per kinase
# If you want to use the full structure set, 
# please use the next filtering step
structures_df = structures_df.groupby(
    "kinase.klifs_name"
).head(n_structures_per_kinase)

structures_df.groupby("kinase.klifs_name").size()

In [ ]:
structure_klifs_ids = structures_df["structure.klifs_id"].to_list()
structure_klifs_ids = [int(i) for i in structure_klifs_ids]
print("Structure KLIFS IDs:", *structure_klifs_ids)

## Kinase similarity: KLIFS IFPs

### Encode structures: Get KLIFS IFPs

- KLIFS IFP: For each kinase structure that is co-crystallized with a ligand, all interactions between the $85$ KLIFS pocket residues and the ligand are described using the IFP by Marcou and Rognan ([<i>JCIM</i> (2007), <b>71(1)</b>, 195-207](https://pubs.acs.org/doi/10.1021/ci600342e)).
- The presence of a certain type of interaction (7 in total) results in the type-setting of a “1” in the bit-string; otherwise a “0” is used to indicate the absence of the interaction. 
- This results in a $85 \times 7 = 595$ bit vector. Since the binding site is aligned across all kinases, each bit position in this IFP can be directly compared across all IFPs in KLIFS. This is what we will do in the practical part of this tutorial.

![KLIFS IFP](https://raw.githubusercontent.com/volkamerlab/teachopencadd/master/teachopencadd/talktorials/T026_kinase_similarity_ifp/images/T026_KLIFS_IFP.png)

hydrophobic contact (HYD); face to face aromatic interactions (F−F); face to edge aromatic interactions (F−E); protein H-bond donor (DON); protein H- bond acceptor (ACC); protein cationic interactions (ION+), protein anionic interactions (ION−)

In [ ]:
ifps_df = session.interactions.by_structure_klifs_id(
    structure_klifs_ids=structure_klifs_ids
)
print(f"Number of IFPs: {len(ifps_df)}")
ifps_df.head()

In [ ]:
structures_with_ifps_df = ifps_df.merge(
    structures_df, on="structure.klifs_id", how="inner"
)
print(f"Number of IFPs: {len(structures_with_ifps_df)}")
structures_with_ifps_df.head()

### Compare structures: KLIFS IFPs

We will make a pairwise comparison of the structures' IFP using the Tanimoto/Jaccard distance as implemented in `sklearn.metrics.pairwise_distances`, which uses under the hood the method `scipy.spatial.distance`.

#### Prepare IFPs as `numpy` array

KLIFS deposits the IFP as a string of 0's and 1's. We have to convert the IFPs to an array of boolean vectors (required by `scipy.spatial.distance` to be able to use the Jaccard distance). Each row in this array refers to one IFP, each columns to one of the IFP's features.

In [ ]:
# Format IFP data (structure KLIFS ID and kinase name set as index)
ifp_series = structures_with_ifps_df.set_index(
    ["structure.klifs_id", "kinase.klifs_name"]
)["interaction.fingerprint"]
ifp_series.head()

In [ ]:
# Cast "0" and "1" to boolean False and True
ifp_series = ifp_series.apply(lambda x: [True if i == "1" else False for i in x])
ifp_series.head()

In [ ]:
structure_klifs_id_per_structure = ifp_series.index.get_level_values(0)
kinase_name_per_structure = ifp_series.index.get_level_values(1)
kinase_name_per_structure

In [ ]:
# Convert to numpy array
ifps_array = np.array(ifp_series.to_list())
ifps_array

#### Calculate pairwise Jaccard distances

The Jaccard distance, defined below, is often used in case of binary fingerprints: 

$$
d_J(A,B) = 1 - J(A,B) = 1 - \frac{\mid A \cap B \mid}{\mid A \cup B \mid}.
$$

In [ ]:
from sklearn.metrics import pairwise_distances

structure_distance_matrix_array = pairwise_distances(ifps_array, metric="jaccard")

In [ ]:
# Create DataFrame with structure KLIFS IDs as index/columns
structure_distance_matrix_df = pd.DataFrame(
    structure_distance_matrix_array,
    index=structure_klifs_id_per_structure,
    columns=structure_klifs_id_per_structure,
)
print(f"Structure distance matrix size: {structure_distance_matrix_df.shape}")
print("Show matrix subset:")
structure_distance_matrix_df.iloc[:5, :5]

### Map structure to kinase distance matrix

Note: So far we compared individual structures, but we want to compare kinases (which can be represented by several structures).

First, as an intermediate step, we will create a structure distance matrix but &mdash; instead of labeling the data with structure KLIFS IDs &mdash; we use the corresponding kinase name.

In [ ]:
# Copy distance matrix to kinase matrix
kinase_distance_matrix_df = structure_distance_matrix_df.copy()
# Replace structure KLIFS IDs with the structures' kinase names
kinase_distance_matrix_df.index = kinase_name_per_structure
kinase_distance_matrix_df.columns = kinase_name_per_structure
print("Show matrix subset:")
kinase_distance_matrix_df.iloc[:5, :5]

In this talktorial, we will consider per kinase pair the two structures that show the most similar binding mode for their co-crystallized ligands. Hence, we select the structure pair with the minimum IFP distance as representative for a kinase pair.

In [ ]:
# We unstack the matrix (each pairwise comparison in a single row)
# We group by kinase names (level=[0, 1] ensures that the order of the kinases is ignored)
# We take the minimum value in each kinase pair group
# We unstack the remaining data points
kinase_distance_matrix_df = (
    kinase_distance_matrix_df.unstack().groupby(level=[0, 1]).min().unstack(level=1)
)
kinase_distance_matrix_df.index.name = None
kinase_distance_matrix_df.columns.name = None

In [ ]:
print(
    f"Structure matrix of shape {structure_distance_matrix_df.shape} "
    f"reduced to kinase matrix of shape {kinase_distance_matrix_df.shape}."
)

In [ ]:
# Rename variable so that we can differentiate the IFP kinase distance matrix
# from the KiSSIm kinase distance matrix that we generate later in this notebook
ifp_kinase_distance_matrix_df = kinase_distance_matrix_df
ifp_kinase_distance_matrix_df

In [ ]:
# Show matrix with background gradient
cm = sns.light_palette("green", as_cmap=True)
ifp_kinase_distance_matrix_df.style.background_gradient(cmap=cm).format("{:.3f}")

## Kinase similarity: KiSSim fingerprints

### Encode structures: Get KiSSim fingerprints

![KiSSim fingerprint](https://raw.githubusercontent.com/volkamerlab/kissim/main/docs/_static/kissim_toc.png)

#### Encoding

In [ ]:
from kissim.encoding import FingerprintGenerator

In [ ]:
FingerprintGenerator.from_structure_klifs_ids?

In [ ]:
%%time
fingerprint_generator = FingerprintGenerator.from_structure_klifs_ids(
    structure_klifs_ids, n_cores=1
)
fingerprint_generator

In [ ]:
# Save fingerprints
json_filepath = "results/fingerprints.json"
fingerprint_generator.to_json(json_filepath)

In [ ]:
fingerprint_generator.data

In [ ]:
example_id1 = list(fingerprint_generator.data)[0]
example_kinase1 = fingerprint_generator.data[example_id1].kinase_name
example_id2 = list(fingerprint_generator.data)[-1]
example_kinase2 = fingerprint_generator.data[example_id2].kinase_name
print(f"Example structure {example_id1} for kinase {example_kinase1}")
print(f"Example structure {example_id2} for kinase {example_kinase2}")

In [ ]:
fp = fingerprint_generator.data[example_id1]
fp

In [ ]:
fp.physicochemical

#### Normalization

In [ ]:
from kissim.encoding import FingerprintGeneratorNormalized

In [ ]:
FingerprintGeneratorNormalized.from_fingerprint_generator?

In [ ]:
%%time
fingerprint_generator_normalized = FingerprintGeneratorNormalized.from_fingerprint_generator(
    fingerprint_generator
)
fingerprint_generator_normalized

In [ ]:
fingerprint_generator_normalized.data

In [ ]:
fingerprint_generator_normalized.data[example_id1].physicochemical

### Compare structures: KiSSim fingerprint

In [ ]:
from kissim.comparison import FingerprintDistanceGenerator

In [ ]:
FingerprintDistanceGenerator.from_fingerprint_generator?

In [ ]:
%%time
fingerprint_distance_generator = FingerprintDistanceGenerator.from_fingerprint_generator(
    fingerprint_generator_normalized, n_cores=1
)
fingerprint_distance_generator

In [ ]:
fingerprint_distance_generator.data

### Map structure to kinase distance matrix

In [ ]:
kissim_kinase_distance_matrix_df = fingerprint_distance_generator.kinase_distance_matrix()
kissim_kinase_distance_matrix_df.index.name = None
kissim_kinase_distance_matrix_df.columns.name = None
kissim_kinase_distance_matrix_df

In [ ]:
# Show matrix with background gradient
cm = sns.light_palette("green", as_cmap=True)
kissim_kinase_distance_matrix_df.style.background_gradient(cmap=cm).format("{:.3f}")

In [ ]:
from kissim.viewer import StructureViewer
viewer1 = StructureViewer.from_structure_klifs_id(example_id1)

In [ ]:
viewer1.show()

In [ ]:
from kissim.viewer import StructurePairViewer
viewer2 = StructurePairViewer.from_structure_klifs_ids(example_id1, example_id2)

In [ ]:
viewer2.show()

## Compare IFP and KiSSim similarity

In [ ]:
from scipy.cluster import hierarchy
from scipy.spatial import distance

Let's define two functions to plot a similarity heatmap and a dendrogram.

In [ ]:
def heatmap(score_df, ax=None, title=""):
    """
    Generate a heatmap from a matrix.

    Parameters
    ----------
    score_df : pd.DataFrame
        Distance or similarity score matrix.
    ax : matplotlib.axes
        Plot axis to use!
    title : str
        Plot title.
    """
    sns.heatmap(score_df, linewidths=0, annot=True, square=True, cmap="viridis", ax=ax)

In [ ]:
def dendrogram(distance_matrix, ax=None, title=""):
    """
    Generate a dendrogram from a distance matrix.

    Parameters
    ----------
    distance_matrix : pd.DataFrame
        Distance matrix.
    ax : matplotlib.axes
        Plot axis to use!
    title : str
        Plot title.
    """
    D = distance_matrix.values
    D_condensed = distance.squareform(D)
    hclust = hierarchy.linkage(D_condensed, method="average")
    tree = hierarchy.to_tree(hclust)
    labels = distance_matrix.columns.to_list()
    hierarchy.dendrogram(hclust, labels=labels, orientation="left", ax=ax)
    ax.set_title(title)
    ax.set_xlabel("Distance")

Define a dictionary containing different similarity measures (perspective) and the respective kinase distance matrices; in our case we look at the KLIFS IFP and KiSSim.

In [ ]:
kinase_distance_matrices_normalized = {
    "KLIFS IFP": ifp_kinase_distance_matrix_df,
    "KiSSim": kissim_kinase_distance_matrix_df,
}

In [ ]:
n_perspectives = len(kinase_distance_matrices_normalized)

fig, axes = plt.subplots(2, n_perspectives, figsize=(n_perspectives * 5, 8))
for i, (perspective, distance_matrix) in enumerate(kinase_distance_matrices_normalized.items()):
    # Heatmap based on similarity matrix
    similarity_matrix = 1 - distance_matrix
    heatmap(similarity_matrix, ax=axes[0][i], title=perspective)
    # Dendrogram based on distance matrix
    dendrogram(distance_matrix, ax=axes[1][i], title=perspective)

Note for ease of interpretability, we show below:

* Heatmaps based on the **similarity** matrix (the higher the value, the higher the similarity).
* Dendrograms calculated based on the **distance** matrix, where clusters describe the similarity between kinases.

## More kinase similarity assessments

https://projects.volkamerlab.org/teachopencadd/talktorials.html#kinase-similarity